In [91]:
import requests
import time

s_time = time.monotonic()
with open("./testfastwarcFULLTEST.warc.gz", "wb") as file:
    response = requests.get("https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/segments/1695233506045.12/warc/CC-MAIN-20230921210007-20230922000007-00501.warc.gz", headers={"Range": "bytes=188657-215140, 29110386-29117436, 29118602-29125637"})
    file.write(response.content)

print(f"Time taken: {time.monotonic() - s_time} ({(time.monotonic() - s_time)} req/sec)")


Time taken: 0.625 (0.625 req/sec)


In [98]:
import time


session = requests.Session()

ranges = [
    "188657-215140",
    "29110386-29117436",
    "29118602-29125637"
]

s_time = time.monotonic()
for range in ranges:
    with session.get("https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-40/segments/1695233506045.12/warc/CC-MAIN-20230921210007-20230922000007-00501.warc.gz", headers={"Range": f"bytes={range}"}) as response:
        with open(f"{range}.warc.gz", "wb") as file:
            file.write(response.content)

print(f"Time taken: {time.monotonic() - s_time} ({(time.monotonic() - s_time) / len(ranges)} req/sec)")

session.close()



Time taken: 0.3440000000009604 (0.11466666666698681 req/sec)
http://103.233.56.101:5003/profiling/main/mudkechula
http://lib.epoka.edu.al/cgi-bin/koha/opac-search.pl?q=ccl=pb%253ABlackwell%252C%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520su-to%253AHydraulics.%2520and%2520holdingbranch%253AEUL%2520and%2520su-to%253AHydraulic%2520engineering.%2520and%2520su-to%253AHydraulic%2520engineering.%2520and%2520itype%253ABK%2520and%2520holdingbranch%253AEUL&sort_by=relevance_dsc&limit=available
http://lib.epoka.edu.al/cgi-bin/koha/opac-search.pl?q=ccl=pb%253ABlackwell%252C%2520and%2520su-to%253AHydraulics.%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520itype%253ABK%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520itype%253ABK%2520and%2520holdingbranch%253AEUL%2520and%2520itype%253ABK

In [101]:
from warcio.archiveiterator import ArchiveIterator

ws_time = time.monotonic()

for range in ranges:
    with open(f"{range}.warc.gz", "rb") as file:
        for warc in ArchiveIterator(file):
            print(warc.rec_headers.get_header("WARC-Target-URI"))

print(f"Time taken: {time.monotonic() - ws_time} ({(time.monotonic() - ws_time) / len(ranges)} req/sec)")

http://103.233.56.101:5003/profiling/main/mudkechula
http://lib.epoka.edu.al/cgi-bin/koha/opac-search.pl?q=ccl=pb%253ABlackwell%252C%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520su-to%253AHydraulics.%2520and%2520holdingbranch%253AEUL%2520and%2520su-to%253AHydraulic%2520engineering.%2520and%2520su-to%253AHydraulic%2520engineering.%2520and%2520itype%253ABK%2520and%2520holdingbranch%253AEUL&sort_by=relevance_dsc&limit=available
http://lib.epoka.edu.al/cgi-bin/koha/opac-search.pl?q=ccl=pb%253ABlackwell%252C%2520and%2520su-to%253AHydraulics.%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520itype%253ABK%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520au%253ANalluri%252C%2520C.%2520and%2520au%253AFeatherstone%252C%2520R.%2520E.%2520and%2520itype%253ABK%2520and%2520holdingbranch%253AEUL%2520and%2520itype%253ABK&sort_by=relevance_dsc&limit=available
Time taken: 0.0 (0.0 r

In [100]:
from fastwarc.warc import ArchiveIterator

fw_time = time.monotonic()

for range in ranges:
    with open(f"{range}.warc.gz", "rb") as file:
        for warc in ArchiveIterator(file):
            print(warc.rec_headers.get_header("WARC-Target-URI"))

print(f"Time taken: {time.monotonic() - fw_time} ({(time.monotonic() - fw_time) / len(ranges)} req/sec)")

AttributeError: 'fastwarc.warc.WarcRecord' object has no attribute 'rec_headers'

In [70]:
count = 0

# outfiles = [open(f"{i}.warc.gz", "wb") for i in range(4)]
outfile = open("./0.warc.gz", "wb")

with open("./testfastwarcFULL.warc.gz", "rb") as file:
    skip_rem = 0
    for line in file:
        line = line.strip()

        if skip_rem > 0:
            skip_rem -= 1
            continue

        if "--CloudFront:B3EEF02223A9CB1F4571FDEA5C5CF183".encode("utf-8") in line:
            # skip forward to after the boundary and headers, then write the rest of the file
            skip_rem = 3
            count += 1
            continue

        outfile.write(line + b"\n")


In [78]:
# from warcio.archiveiterator import ArchiveIterator
from fastwarc.warc import ArchiveIterator

with open("./1.warc.gz", "rb") as stream:
    for warc in ArchiveIterator(stream, strict_mode=False):
        # html = warc.content_stream().read().decode("utf-8")
        print(warc.content_length)

StreamError: Not a valid WARC stream

In [54]:
import io
from warcio.archiveiterator import ArchiveIterator

def print_warcs(stream: io.BytesIO):
    # print("Record!")
    for warc in ArchiveIterator(stream):
        html = warc.content_stream().read()
        print(html[:100])

with open("./testfastwarcFULL.warc", "rb") as stream:
    # for warc in ArchiveIterator(stream):
    #     html = warc.content_stream().read()
    #     print(html)
    buffer = b""

    for line in stream:
        line = line.strip()

        if "--CloudFront:".encode("utf-8") in line:
            if buffer:
                with open("./testfastwarcFULLTEST.warc", "wb") as file:
                    file.write(buffer)
                # print(buffer)
                # print_warcs(io.BytesIO(buffer))
                buffer = b""
            
            continue

        buffer += line

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x00\xec\xbd\xedr\x1b\xc7\x92(\xf8\xdb\x8e8\xef\xd0\x97\x133\xd7\xbe#\x90\xf8\x06)K\x9a%\x08\x89")R4\x01YG\xde\xd88Q@\x17\xd1\x05\xf4\xd7\xe9\x0fRp\xdc\x1f\xfb\x1a\xfb\x003\xbbswc\xef+M\xec\x93leVu\x03 A\xa4\x91L\xd9#\xd8"\xb2\x80FWuUVV~\xe7\xc7\xfd\x8b\x83\x9d\xdav\xf5/\xdf\x7f\xd4Pe0\x8b\xe5s\'\x91i\x1c\x85\xa9\xb4\x1f\xf6D\xa6?\xacW\xeb\x8dJu\xafR\xaf\r\xea\x8d\xe7\xb5\xf6\xf3V\xedW{\xc1\x85\x1cE\x89[9\xea=w^\xe4I\xf8<\xcf\x95\xfb\xbc>\xac\x89z\xc3\xedTF\xcdN\xa3\xd2\xec\xb4\xaa\x95\xdd\xcb\x86\xacT/\x87\xc3Fu\xb7\xd6v/\xab\xaf\xfe\xf2\xfdA\x14f2\xcc*\xefd8\xce<\xddM\xabQo\xef\xcd?7#\x12q\xec\xab\x91\xc8T\x14\xeexY\x16\xff\xe4\x04\xe98\xd3_\xbd\xbc1\xd6\x8f"\x19\xa9\xf02\xba1\x18\xd9\x11Uq\xd9nU\xea\xd5\x9a\xac4u\x0f\x15\xb1\xd7\x16\x15\xb9\xdb\xaa\xee\xb6\xdc\xdd\xfa^\xb3\xf1\xca\xdeB\xf7<\xca\x93\x04;\x8f\x16o\xb2\xd7\xeeT\x9bM1\xac\xb8\xd5\xdda\xa5\xd9n5+\xc3\xbdQ\xb3\xd2\x91\xbb\xf5Z\xad\xd3\xa9\xb7[\xa2\xb8\xc9\xd1ye\xdfu\xf5\xe0\xd2\xe7N\x